In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
# 输入研究区的shp文件所在路径
wuhan_shp ="D:/NDVI_exercise/wuhan/wuhanregion.shp"
# shp 转 ee 
wuhan_ee = geemap.shp_to_ee(wuhan_shp)
# 添加到layer
# Map.addLayer(wuhan_ee,{},"wuhan")
Map.centerObject(wuhan_ee,7)   # 设置以研究区居中

In [4]:
# 去云掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)
# landsat8 图像可视化参数
visParams = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}
# JRC数据集的可视化参数
visualization = {
    'bands': ['waterClass'],
    'min': 0.0,
    'max': 3.0,
    'palette': ['cccccc', 'ffffff', '99d9ea', '0000ff']
}
# NDVI and mNDWI 函数的定义
def NDVI(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')  
    return image.addBands(ndvi)

def mNDWI(image):
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    return image.addBands(mndwi)

# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# 分类标签
label = 'waterclass'


In [5]:
# 添加一个底图白板
basemap = ee.Image.constant(0).clip(wuhan_ee).select('constant').rename('waterclass')
# Map.addLayer(basemap,{'palette':['white']},"basemap")

In [ ]:
# dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
#                   .filterBounds(wuhan_ee) \
#                   .filterDate('2018-01-01', '2018-12-30') \
#                   .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5))

# visualization = {
#   'min': 0.0,
#   'max': 3500,
#   'bands': ['B8', 'B4', 'B3'],
# }

# Map.addLayer(dataset.first().clip(wuhan_ee), visualization, 'RGB')

# 训练&分类

In [6]:
landsat8_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(wuhan_ee) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr)

landsat_ndvi_mndwi = landsat8_images.map(NDVI).map(mNDWI)
# 计算有效像元的个数
validPixel = landsat8_images.count().select('B3').clip(wuhan_ee).rename('count')
# Map.addLayer(validPixel,{},'count_image')

In [ ]:
# from matplotlib import pyplot as plt
# import numpy as np
# import matplotlib
# from geemap import cartoee
# region = [113.6, 29.9, 115.1, 31.4] 
# vis = {'min':0, 'max':40}
# # vis = {'bands': ['count'], 'palette': ['ffffff', 'dd09ec'], 'min': 0.0, 'max': 40.0, 'opacity': 1.0}
# fig = plt.figure(figsize=(12, 8))
# cmap = 'YlGn'
# # use cartoee to get a map
# ax = cartoee.get_map(validPixel, region=region, vis_params=vis,cmap = cmap)
# # add a colorbar to the map using the visualization params we passed to the map
# cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="number of valid pixels", orientation="vertical")
# # ticks=range(0,50,10)
# # add gridlines to the map at a specified interval
# cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
# ax.set_title(label = 'number of valid pixels in 2018', fontsize=20)
# # plt.show()


In [7]:
JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(wuhan_ee)
JRC_permanentwater = JRC_wuhan_water.eq(3).select("waterClass").rename('jrcwaterclass')
# Map.addLayer(JRC_permanentwater.selfMask(),{'palette':['blue']},'jrcpermanent')
JRC_samplepoints = JRC_permanentwater.sample(**{
    'region': wuhan_ee,
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

# 70% 做训练点位，30% 做验证点位
withRandom = JRC_samplepoints.randomColumn('random')

#把生成随机数大于0.7的分成测试数据，否则为训练数据。这样可以把数据分成两份
split = 0.7
trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
testingPartition = withRandom.filter(ee.Filter.gte('random', split))
# print(testingPartition.first().getInfo())
# print('JRC total number of sample point:{}'.format(testingPartition.size().getInfo()))
# # print(JRC_samplepoints.first().getInfo())
# waterclass = testingPartition.filter(ee.Filter.eq('jrcwaterclass',1))
# print('the number of water sample point:{}'.format(waterclass.size().getInfo()))

In [8]:
def imageSample(image):
    # Overlay the points on the imagery to get training.
    sample_point = image.select(bands).sampleRegions(**{
        'collection': trainingPartition,
        'properties': ['jrcwaterclass'],
        'scale': 30
    })
    return sample_point
points_collection = landsat_ndvi_mndwi.map(imageSample).flatten()
# print('total sample number:{}'.format(points_collection.size().getInfo()))
# print(points_collection.first().getInfo())
# samplepoint_water = points_collection.filter(ee.Filter.eq('jrcwaterclass',1))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))
# print(samplepoint_water.first().getInfo())

In [9]:
trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,'jrcwaterclass',bands)
#对Landsat-8进行分类
def training(image):
    return image.clip(wuhan_ee).select(bands).classify(trainedClassifier).eq(1).selfMask().select('classification').rename('waterclass')
landsatImage_classify = landsat_ndvi_mndwi.map(training)
waterpixel_count = landsatImage_classify.sum()
# print(landsatImage_classify.size().getInfo())
# Map.addLayer(waterpixel_count,{},"waterPixelCount")
wuhan_waterfrequency = waterpixel_count.select('waterclass').divide(validPixel.select('count')).select('waterclass').rename('frequency')
wuhan_permanentwater = wuhan_waterfrequency.gte(0.75).selfMask().select('frequency').rename('waterclass')

# Map.addLayer(wuhan_waterfrequency,{},"waterfrequency")
# Map.addLayer(wuhan_permanentwater,{'palette':['red']},"permanentwater")

In [ ]:
# from geemap import cartoee
# region = [113.6, 29.9, 115.1, 31.4] 
# vis = {'min':0, 'max':40}
# fig = plt.figure(figsize=(12, 8))
# cmap = 'GnBu'
# # use cartoee to get a map
# ax = cartoee.get_map(waterpixel_count, region=region, vis_params=vis,cmap = cmap)
# # add a colorbar to the map using the visualization params we passed to the map
# cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="number of water pixels", orientation="vertical")
# # ticks=range(0,50,10)
# # add gridlines to the map at a specified interval
# cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
# ax.set_title(label = 'number of water pixels in 2018', fontsize=20)

In [ ]:
# from geemap import cartoee
# region = [113.6, 29.9, 115.1, 31.4] 
# vis = {'min':0, 'max':1}
# # vis = {'bands': ['count'], 'palette': ['ffffff', 'dd09ec'], 'min': 0.0, 'max': 40.0, 'opacity': 1.0}
# fig = plt.figure(figsize=(12, 8))
# cmap = 'Blues'
# # use cartoee to get a map
# ax = cartoee.get_map(wuhan_waterfrequency, region=region, vis_params=vis,cmap = cmap)
# # add a colorbar to the map using the visualization params we passed to the map
# cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
# # ticks=range(0,50,10)
# # add gridlines to the map at a specified interval
# cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
# ax.set_title(label = 'waterbody frequency in 2018', fontsize=20)

In [ ]:
# from geemap import cartoee
# region = [113.6, 29.9, 115.1, 31.4] 
# vis = {'bands': ['waterclass'], 'palette': ['blue'], 'min': 0.0, 'max': 1.0, 'opacity': 1.0}
# fig = plt.figure(figsize=(12, 8))
# # use cartoee to get a map
# ax = cartoee.get_map(wuhan_permanentwater, region=region, vis_params=vis)
# # add gridlines to the map at a specified interval
# cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
# ax.set_title(label = 'permanent surface water in 2018(JRC)', fontsize=20)

In [ ]:
# from geemap import cartoee
# region = [113.6, 29.9, 115.1, 31.4] 
# vis = {'bands': ['jrcwaterclass'], 'palette': ['blue'], 'min': 0.0, 'max': 1.0, 'opacity': 1.0}
# fig = plt.figure(figsize=(12, 8))
# # use cartoee to get a map
# ax = cartoee.get_map(JRC_permanentwater.selfMask(), region=region, vis_params=vis)
# # add gridlines to the map at a specified interval
# cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
# ax.set_title(label = 'JRC permanent surface water in 2018', fontsize=20)

# 验证

In [10]:
imagecollection = ee.ImageCollection([wuhan_permanentwater,basemap])
permanent_water_layer = imagecollection.sum()
# Map.addLayer(permanent_water_layer,{'min':0,"max":1,'palette':['white','blue']},'waterpermanent')

In [12]:
# # 使用JRC做验证
# JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/2018").clip(wuhan_ee)
# Map.addLayer(JRC_wuhan_water,visualization,'JRC_wuhan_water')
# JRC_permanentwater = JRC_wuhan_water.eq(3).select("waterClass").rename('jrcwaterclass')
# JRC_samplepoints = JRC_permanentwater.sample(**{
#     'region': wuhan_ee,
#     'scale': 30,
#     'numPixels': 8 000,
#     'seed': 0,
#     'geometries': True  # Set this to False to ignore geometries
# })
# Map.addLayer(JRC_samplepoints,{},'JRC_samplepoints')
# print('JRC total number of sample point:{}'.format(JRC_samplepoints.size().getInfo()))
# # print(JRC_samplepoints.first().getInfo())
# waterclass = JRC_samplepoints.filter(ee.Filter.eq('jrcwaterclass',1))
# print('the number of water sample point:{}'.format(waterclass.size().getInfo()))
# Overlay the points on the imagery to get training.
print(testingPartition.size().getInfo())
JRC_sample_point = permanent_water_layer.sampleRegions(**{
    'collection': testingPartition,
    'properties': ['jrcwaterclass'],
    'scale': 30
})
print(JRC_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = JRC_sample_point.errorMatrix('jrcwaterclass', 'waterclass')
print(test_accuracy.getInfo())
print('total caccuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

366
{'type': 'Feature', 'geometry': None, 'id': '1_0', 'properties': {'jrcwaterclass': 0, 'waterclass': 0}}
[[230, 4], [23, 109]]
total caccuracy:0.9262295081967213
kappa:0.8348541060864334


In [15]:
imagecollection = ee.ImageCollection([wuhan_permanentwater,basemap])
permanent_water_layer = imagecollection.sum()
# Map.addLayer(permanent_water_layer,{'min':0,"max":1,'palette':['white','blue']},'waterpermanent')

# 使用Maryland做验证
basemap1 = ee.Image.constant(0).clip(wuhan_ee).select('constant').rename('water_class')
Maryland_2018 = ee.Image('users/311605001111/Maryland_2018').clip(wuhan_ee).select('b1').gt(74).select('b1').rename('water_class')
wuhan_Maryland = ee.ImageCollection([Maryland_2018,basemap1]).sum()
Maryland_samplepoints =wuhan_Maryland .sample(**{
    'region': wuhan_ee,
    'scale': 30,
    'numPixels': 3000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})
print('Maryland total number of sample point:{}'.format(Maryland_samplepoints.size().getInfo()))
# print(Maryland_samplepoints.first().getInfo())
waterclass = Maryland_samplepoints.filter(ee.Filter.eq('water_class',1))
print('the number of water sample point:{}'.format(waterclass.size().getInfo()))
# Overlay the points on the imagery to get training.
Maryland_sample_point = permanent_water_layer.sampleRegions(**{
    'collection': Maryland_samplepoints,
    'properties': ['water_class'],
    'scale': 30
})
print(Maryland_sample_point.first().getInfo())
# 利用误差矩阵进行验证
test_accuracy = Maryland_sample_point.errorMatrix('water_class', 'waterclass')
print(test_accuracy.getInfo())
print('total caccuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

Maryland total number of sample point:2999
the number of water sample point:360
{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'water_class': 0, 'waterclass': 0}}
[[2634, 5], [130, 230]]
total caccuracy:0.9549849949983328
kappa:0.7493414148668436


# 水体面积

In [ ]:
# calculate the area of permanent water
waterarea = wuhan_permanentwater.multiply(ee.Image.pixelArea()).divide(1e6)
areas = waterarea.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': wuhan_ee.geometry(),
    'scale': 30,
    'maxPixels': 1e14
})
print(areas.getInfo())

In [ ]:
vis_params = {'bands': ['count'], 'palette': ['ffffff', ' dd09ec'], 'min': 0.0, 'max': 40.0, 'opacity': 1.0, 'gamma': 1.0}

In [ ]:
import ee
import geemap
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
Map = geemap.Map()
Map